In [7]:
import sys
sys.path.append('/uio/kant/geo-metos-u7/franzihe/Documents/Thesis/Python')

import numpy as np
import netCDF4
import pandas as pd
import matplotlib.pyplot as plt

import createFolder as cF
np.warnings.filterwarnings('ignore')



In [8]:
m = ['11', '12', '01', '02', '03']
station = 'Haukeliseter'

### Retrieval data

In [9]:
txt_dir = '../../Data/Retrieval/'
txt_filename = 'pulsed_minute_surface_acc_Haukeli.dat.txt'
txt_filename = 'upslope_minute_surface_acc_Haukeli.dat.txt'

In [10]:
def get_netCDF_variable(f, var_name, var, dim):
    v_0m = f.createVariable(varname=var_name, datatype=var.dtype, dimensions=dim, zlib=True)
    v_0m[:] = var[:]
    return(v_0m)

In [11]:
Haukeli = pd.read_fwf(txt_dir+txt_filename,header=1)

In [ ]:
for i in range(int(np.round((Haukeli.shape[0]-1)/int(1440/6+1)))):
    Date = int(Haukeli['Unnamed: 0'][i*int(1440/6+1)])
    precip_24 = (Haukeli['Unnamed: 1'][i*int(1440/6+1)])
    precip_24 = np.array([precip_24])
    
    column0 = Haukeli['Unnamed: 0'][i*int(1440/6+1)+1:(i+1)*int(1440/6+1)]
    column1 = Haukeli['Unnamed: 1'][i*int(1440/6+1)+1:(i+1)*int(1440/6+1)]
    column2 = Haukeli['Unnamed: 2'][i*int(1440/6+1)+1:(i+1)*int(1440/6+1)]
    column3 = Haukeli['Unnamed: 3'][i*int(1440/6+1)+1:(i+1)*int(1440/6+1)]
    column4 = Haukeli['Unnamed: 4'][i*int(1440/6+1)+1:(i+1)*int(1440/6+1)]
    column5 = Haukeli['Unnamed: 5'][i*int(1440/6+1)+1:(i+1)*int(1440/6+1)]

    column0 = column0.convert_objects(convert_numeric=True)
    column1 = column1.convert_objects(convert_numeric=True)
    column2 = column2.convert_objects(convert_numeric=True)
    column3 = column3.convert_objects(convert_numeric=True)
    column4 = column4.convert_objects(convert_numeric=True)
    column5 = column5.convert_objects(convert_numeric=True)

    obs_diff = pd.concat([column0, column1, column2, column3, column4, column5], axis =1)
    obs_diff = np.array(obs_diff)

    obs_diff = obs_diff.flatten()/obs_diff.flatten().shape[0]
    

In [12]:
    filename = txt_filename.split('_')[0]+'_precip_%s.nc' %(Date)

    ### write netCDF file
    fw = netCDF4.Dataset(txt_dir+filename, 'w')
    ### create dimensions
    fw.createDimension('time', obs_diff.shape[0])
    fw.createDimension('24h_acc_sum', precip_24.shape[0])
    
    precip_w       = get_netCDF_variable(fw, 'precipitation', obs_diff, ('time',))
    total_precip_w = get_netCDF_variable(fw, '24h_precip_acc', precip_24, ('24h_acc_sum',) )
    
    fw.close()